In [ ]:
# Earnings (today → +3 days) – IV / HV • Yahoo Screener • multithread • Jupyter
# ---------------------------------------------------------------------------------
# Kullanım:
# 1. Aşağıdaki **Gerekli Paketler** listesini kendi ortamınıza _manuel_ yükleyin.
#    Sandbox veya kısıtlı ortamlarda `subprocess` kullanılamadığından kod
#    otomatik `pip install` yapmaz.
# 2. Bu hücreyi Jupyter Notebook / JupyterLab içine *tek parça* yapıştırıp
#    çalıştırın. Çalışma sürecini durdurmak için hücreyi «Interrupt/Stop» edin.
#
# Gerekli Paketler
# ----------------
#   yfinance>=0.2.38
#   pandas>=1.4
#   numpy>=1.23
#   requests>=2.28
#   beautifulsoup4>=4.12
#   lxml>=4.9
#   tqdm>=4.66  (ipywidgets>=7 önerilir – yüklü değilse metin moduna döner)
#   websockets>=11  (yfinance bağımlılığı)
#
# Önemli Notlar
# -------------
# • Kod _otomatik kurulum_ yapmaz; eksik modül varsa temiz bir ImportError verir.
# • Eğer ipywidgets yoksa `tqdm` progress bar'ı metin modunda görünür.
# • `os.environ["YF_NO_CURL"] = "1"` ile yfinance'ın `curl_cffi` yolunu
#   kapatıyoruz, böylece Ctrl‑C/Interrupt sorunsuz çalışıyor.

import os, time, logging, concurrent.futures, requests
from datetime import date, timedelta
from math import sqrt

# --- 3.‑parti modüller -------------------------------------------------------
try:
    import yfinance as yf
    import pandas as pd
    import numpy as np
    from bs4 import BeautifulSoup
    from IPython.display import clear_output, display
    from tqdm.auto import tqdm
except ImportError as e:
    missing = str(e).split("'")[1]
    raise ImportError(
        f"Gerekli paket '{missing}' yüklü değil. Lütfen yukarıdaki listeye göre pip install yapın.")

# --- Genel ayarlar -----------------------------------------------------------
os.environ["YF_NO_CURL"] = "1"              # Ctrl‑C uyumluluğu
logging.getLogger("yfinance").setLevel(logging.ERROR)
TODAY       = date.today()
LIMIT_DATE  = TODAY + timedelta(days=3)
REFRESH_SEC = 60
THREADS     = min(32, (os.cpu_count() or 4) * 2)

# --- 1. Opsiyonlu sembolleri çek (Yahoo Screener) ---------------------------

def get_optionable_symbols(max_pages: int = 120, pause: float = 0.4):
    """Yahoo Screener 'optionable' listesini getirir (≈ 4 000 sembol)."""
    url_tpl = (
        "https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved"
        "?scrIds=optionable&count=100&start={}"
    )
    headers = {"User-Agent": "Mozilla/5.0"}
    symbols = []

    for page in range(max_pages):
        start = page * 100
        resp = requests.get(url_tpl.format(start), headers=headers, timeout=10)
        # JSON bekleniyor; değilse sayfayı atla
        if not resp.headers.get("Content-Type", "").startswith("application/json"):
            time.sleep(pause)
            continue
        try:
            js = resp.json() or {}
            quotes = ((js.get("finance") or {})
                      .get("result", [{}])[0]
                      .get("quotes", []))
            if not quotes:
                break  # sayfa bitti
            symbols.extend(q.get("symbol") for q in quotes if q.get("symbol"))
        except (ValueError, TypeError):
            pass  # bozuk JSON – sayfayı atla
        time.sleep(pause)  # rate‑limit
    return symbols

OPTIONABLE = get_optionable_symbols()
print(f"Optionable universe size: {len(OPTIONABLE)}")

# --- 2. Earnings tarama (paralel) -------------------------------------------

def next_earn_ts(sym: str):
    t = yf.Ticker(sym)
    try:
        if hasattr(t, "get_next_earnings_date"):
            ts = t.get_next_earnings_date()
            if pd.notna(ts):
                return sym, ts
        ed = t.get_earnings_dates(limit=6)
        fut = ed[ed.index.date >= TODAY]
        return (sym, fut.index[0]) if not fut.empty else (sym, None)
    except Exception:
        return sym, None


def earnings_window(symbols):
    keep: dict[str, pd.Timestamp] = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=THREADS) as ex:
        futures = {ex.submit(next_earn_ts, s): s for s in symbols}
        for f in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="earnings scan"):
            sym, ts = f.result()
            if ts is not None and TODAY <= ts.date() <= LIMIT_DATE:
                keep[sym] = ts
    return keep

# --- 3. IV / HV hesapları ----------------------------------------------------

def atm_iv(sym: str):
    try:
        t = yf.Ticker(sym)
        exp  = sorted(t.options)[0]
        oc   = t.option_chain(exp)
        spot = t.fast_info.get("lastPrice") or t.history(period="1d")["Close"].iloc[-1]
        calls = oc.calls.assign(d=lambda d: abs(d.strike - spot))
        puts  = oc.puts .assign(d=lambda d: abs(d.strike - spot))
        civ   = calls.sort_values("d").iloc[0]["impliedVolatility"]
        piv   = puts .sort_values("d").iloc[0]["impliedVolatility"]
        if pd.isna(civ) or pd.isna(piv):
            return None
        return (civ + piv) / 2
    except Exception:
        return None


def hist_vol(sym: str, win: int = 30):
    try:
        px = yf.download(sym, period=f"{win*2}d", progress=False)["Close"].tail(win + 1)
        if len(px) < win + 1:
            return None
        return np.log(px / px.shift(1)).dropna().std(ddof=0) * sqrt(252)
    except Exception:
        return None

# --- 4. Canlı döngü ---------------------------------------------------------
try:
    while True:
        clear_output(wait=True)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}]  Earnings (today → +3d)  –  IV / HV\n")

        upcoming = earnings_window(OPTIONABLE)  # {sym: ts}

        records = []
        for sym, ts in tqdm(upcoming.items(), desc="IV/HV calc"):
            iv = atm_iv(sym)
            hv = hist_vol(sym)
            if iv is None or hv in (None, 0):
                continue
            records.append({
                "Earnings": ts.strftime("%d-%m-%Y %H:%M"),
                "Ticker"  : sym,
                "30d IV"  : f"{iv*100:5.1f}%",
                "30d HV"  : f"{hv*100:5.1f}%",
                "IV/HV"   : f"{iv/hv:4.2f}",
            })

        cols = ["Earnings", "Ticker", "30d IV", "30d HV", "IV/HV"]
        df = pd.DataFrame(records, columns=cols)
        if not df.empty:
            df = df.sort_values("Earnings").reset_index(drop=True)
        display(df if not df.empty else pd.DataFrame({"info": ["No optionable earnings in window."]}))

        time.sleep(REFRESH_SEC)
except KeyboardInterrupt:  # hücre «Interrupt» edildiğinde
    clear_output()
    print("Stopped.")


[2025-05-14 10:24:04]  Earnings (today → +3d)  –  IV / HV



earnings scan: 0it [00:00, ?it/s]
IV/HV calc: 0it [00:00, ?it/s]


,info
0,No optionable earnings in window.


In [ ]:
# -- Optionable symbol list (Yahoo Screener) ---------------------------------
# Gereken paketler: requests, pandas
import requests, pandas as pd, time, json

def get_optionable_symbols(max_pages: int = 120, pause: float = 0.3) -> list[str]:
    """
    Yahoo screener 'optionable' sonuçlarını 100'lük sayfalarda indirir.
    max_pages * 100 kayda kadar tarar, JSON gelmeyen sayfaları atlar.
    """
    base = ("https://query2.finance.yahoo.com/v1/finance/screener/predefined/saved"
            "?scrIds=optionable&count=100&start={}")
    hdrs = {"User-Agent": "Mozilla/5.0"}
    
    symbols: list[str] = []
    for page in range(max_pages):
        start = page * 100
        resp = requests.get(base.format(start), headers=hdrs, timeout=10)

        if resp.headers.get("Content-Type", "").startswith("application/json"):
            try:
                js = resp.json()
                quotes = (
                    js.get("finance", {})
                      .get("result", [{}])[0]
                      .get("quotes", [])
                )
                # Sayfa boşsa döngüyü bitir
                if not quotes:
                    break
                symbols.extend(q["symbol"] for q in quotes)
            except (ValueError, KeyError, TypeError):
                print(f"[warn] JSON parse failed for start={start}, sayfa atlandı")
        else:
            print(f"[warn] Non-JSON response (start={start}), sayfa atlandı")
        
        time.sleep(pause)                # rate-limit koruması
    
    return symbols

# ---- Hücre çıktısı ----------------------------------------------------------
symbols = get_optionable_symbols()
print(f"Toplam optionable sembol sayısı: {len(symbols)}")
print("İlk 25 sembol:", symbols[:25])


[warn] JSON parse failed for start=0, sayfa atlandı
[warn] JSON parse failed for start=100, sayfa atlandı
[warn] JSON parse failed for start=200, sayfa atlandı
[warn] JSON parse failed for start=300, sayfa atlandı
[warn] JSON parse failed for start=400, sayfa atlandı
[warn] JSON parse failed for start=500, sayfa atlandı
[warn] JSON parse failed for start=600, sayfa atlandı
[warn] JSON parse failed for start=700, sayfa atlandı
[warn] JSON parse failed for start=800, sayfa atlandı
[warn] JSON parse failed for start=900, sayfa atlandı
[warn] JSON parse failed for start=1000, sayfa atlandı


KeyboardInterrupt: 

In [ ]:
# Earnings Dashboard (today → +7 days) — IV / HV  •  From user‑provided ticker file
# =================================================================================
# 👉 KULLANIM
# 1. Ticker listenizi **CSV** dosyası olarak “C:\Users\PC\Desktop\Dokümanlar\Sistem Test\Earnings Plays”
#    klasörüne koyun — ör. `symbols.csv`.
#    * İlk sütun sembolleri içermelidir (başlık adı önemsiz; "Ticker", "Symbol" vb.).
# 2. Aşağıdaki değişkenleri gerekirse düzenleyin:
#       FILE_PATH     : dosyanın tam yolu
#       THREADS       : eşzamanlı iş parçacığı sayısı (IO‑bound → 32 iyi)
#       WINDOW_DAYS   : earnings penceresi (+7)
# 3. Hücreyi Jupyter Notebook / JupyterLab’da tek parça çalıştırın.
# 4. Çıktı otomatik yenilenir (`REFRESH_SEC`).  Durdurmak için Kernel ▸ Interrupt.
#
# ---------------------------------------------------------------------------------
# Gereken paketler (manuel kurun):
#   yfinance >= 0.2.41    pandas    numpy
#   requests    beautifulsoup4    lxml    websockets    tqdm    ipywidgets (opsiyonel)

import os, time, logging, concurrent.futures, requests, yfinance as yf
import pandas as pd, numpy as np
from datetime import date, timedelta
from math import sqrt
from IPython.display import clear_output, display
from tqdm.auto import tqdm

# --------- KONSTANLAR ------------------------------------------------------------------------
FILE_PATH    = r"C:\Users\PC\Desktop\Dokümanlar\Sistem Test\Earnings Plays\symbols.csv"
THREADS      = 32           # ağ‑bound, 32 genelde güvenli
WINDOW_DAYS  = 7            # bugün + N gün penceresi
REFRESH_SEC  = 60           # dashboard yenileme saniyesi

os.environ["YF_NO_CURL"] = "1"          # curl_cffi devre dışı (Ctrl+C uyumlu)
logging.getLogger("yfinance").setLevel(logging.ERROR)

TODAY   = date.today()
LIMIT   = TODAY + timedelta(days=WINDOW_DAYS)

# --------- 1) Kullanıcı dosyasından ticker listesi -------------------------------------------

def load_user_symbols(path: str) -> list[str]:
    df = pd.read_csv(path, nrows=0)  # sadece başlıklar için
    first_col = df.columns[0]
    tickers = pd.read_csv(path, usecols=[first_col])[first_col]
    return (tickers.astype(str)
                  .str.strip()
                  .str.upper()
                  .str.replace(r"[\./]", "-", regex=True)
                  .str.lstrip("$")
                  .dropna()
                  .unique()
                  .tolist())

# --------- 2) yfinance yardımcıları -----------------------------------------------------------

def next_earn_ts(tkr: yf.Ticker):
    """Bugünden sonraki ilk earnings datetime (None → yok)."""
    try:
        if hasattr(tkr, "get_next_earnings_date"):
            ts = tkr.get_next_earnings_date()
            if pd.notna(ts):
                return ts
        ed = tkr.get_earnings_dates(limit=8)
        fut = ed[ed.index.date >= TODAY]
        return fut.index[0] if not fut.empty else None
    except Exception:
        return None

def atm_iv(tkr: yf.Ticker):
    try:
        if not tkr.options:
            return None
        exp  = sorted(tkr.options)[0]
        oc   = tkr.option_chain(exp)
        spot = tkr.history(period="1d")["Close"].iloc[-1]
        calls = oc.calls.assign(d=lambda d: abs(d.strike-spot))
        puts  = oc.puts .assign(d=lambda d: abs(d.strike-spot))
        civ   = calls.sort_values("d").iloc[0]["impliedVolatility"]
        piv   = puts .sort_values("d").iloc[0]["impliedVolatility"]
        if pd.isna(civ) or pd.isna(piv):
            return None
        return (civ + piv)/2
    except Exception:
        return None

def hist_vol(sym: str, win: int = 30):
    try:
        px = yf.download(sym, period=f"{win*2}d", progress=False)["Close"].tail(win+1)
        if len(px) < win+1:
            return None
        return np.log(px/px.shift(1)).dropna().std(ddof=0)*sqrt(252)
    except Exception:
        return None

# --------- 3) Paralel worker ------------------------------------------------------------------

def worker(sym: str):
    tkr = yf.Ticker(sym)

    if not tkr.options:
        return {"status": "non_optionable", "ticker": sym}

    ts = next_earn_ts(tkr)
    if ts is None or not (TODAY <= ts.date() <= LIMIT):
        return {"status": "no_window", "ticker": sym}

    iv = atm_iv(tkr)
    hv = hist_vol(sym)
    if iv is None or hv in (None, 0):
        return {"status": "skip", "ticker": sym}

    try:
        name = tkr.fast_info.get("shortName") or tkr.get_info().get("shortName", "-")
    except Exception:
        name = "-"

    return {
        "status" : "ok",
        "Ticker" : sym,
        "Company": name,
        "Earnings": ts.strftime("%d-%m-%Y %H:%M"),
        "IV/HV"  : f"{iv/hv:4.2f}"
    }

# --------- 4) Canlı dashboard döngüsü ----------------------------------------------------------
try:
    symbols_master = load_user_symbols(FILE_PATH)
    print(f"Dosyadan okunan sembol sayısı: {len(symbols_master)}")
except Exception as e:
    raise RuntimeError(f"Ticker dosyası okunamadı: {e}")

while True:
    clear_output(wait=True)
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}]  Earnings (today → +{WINDOW_DAYS}d) — IV/HV\n")

    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=THREADS) as ex:
        for res in tqdm(ex.map(worker, symbols_master), total=len(symbols_master), desc="scan"):
            results.append(res)

    df_ok = pd.DataFrame([r for r in results if r["status"] == "ok"],
                         columns=["Ticker","Company","Earnings","IV/HV"])
    df_ok = df_ok.sort_values("Earnings").reset_index(drop=True)

    non_opt_count = sum(r["status"] == "non_optionable" for r in results)

    if df_ok.empty:
        display(pd.DataFrame({"info":["Pencerede optionable earnings yok"]}))
    else:
        display(df_ok)

    print(f"\nOptionable olmayan sembol sayısı: {non_opt_count}")
    time.sleep(REFRESH_SEC)
except KeyboardInterrupt:
    clear_output()
    print("Stopped.")

Opsiyonlanabilir hisse senetleri aranıyor...
S&P 500'den 503 adet hisse senedi bulundu. Opsiyon verileri kontrol ediliyor...
MMM opsiyonlanabilir.
AOS opsiyonlanabilir.
ABT opsiyonlanabilir.
ABBV opsiyonlanabilir.
ACN opsiyonlanabilir.
ADBE opsiyonlanabilir.
AMD opsiyonlanabilir.
AES opsiyonlanabilir.
AFL opsiyonlanabilir.
A opsiyonlanabilir.
APD opsiyonlanabilir.
ABNB opsiyonlanabilir.
AKAM opsiyonlanabilir.
ALB opsiyonlanabilir.
ARE opsiyonlanabilir.
ALGN opsiyonlanabilir.
ALLE opsiyonlanabilir.
LNT opsiyonlanabilir.
ALL opsiyonlanabilir.
GOOGL opsiyonlanabilir.
GOOG opsiyonlanabilir.
MO opsiyonlanabilir.
AMZN opsiyonlanabilir.
AMCR opsiyonlanabilir.


Exception ignored from cffi callback <function buffer_callback at 0x00000250E3703A60>:
Traceback (most recent call last):
  File "c:\Users\PC\Documents\GitHub\Portfolio\.venv\Lib\site-packages\curl_cffi\curl.py", line 63, in buffer_callback
    @ffi.def_extern()
KeyboardInterrupt: 


AEE opsiyonlanabilir.
AEP opsiyonlanabilir.
AXP opsiyonlanabilir.
AIG opsiyonlanabilir.
AMT opsiyonlanabilir.
AWK opsiyonlanabilir.
AMP opsiyonlanabilir.
AME opsiyonlanabilir.
AMGN opsiyonlanabilir.
APH opsiyonlanabilir.
ADI opsiyonlanabilir.
ANSS opsiyonlanabilir.
AON opsiyonlanabilir.
APA opsiyonlanabilir.
APO opsiyonlanabilir.
AAPL opsiyonlanabilir.
AMAT opsiyonlanabilir.
APTV opsiyonlanabilir.
ACGL opsiyonlanabilir.
ADM opsiyonlanabilir.
ANET opsiyonlanabilir.
AJG opsiyonlanabilir.
AIZ opsiyonlanabilir.
T opsiyonlanabilir.
ATO opsiyonlanabilir.
ADSK opsiyonlanabilir.
ADP opsiyonlanabilir.
AZO opsiyonlanabilir.
AVB opsiyonlanabilir.
AVY opsiyonlanabilir.
AXON opsiyonlanabilir.
BKR opsiyonlanabilir.
BALL opsiyonlanabilir.
BAC opsiyonlanabilir.
BAX opsiyonlanabilir.
BDX opsiyonlanabilir.
BRK.B için opsiyon bulunamadı.
BBY opsiyonlanabilir.
TECH opsiyonlanabilir.
BIIB opsiyonlanabilir.
BLK opsiyonlanabilir.
BX opsiyonlanabilir.
BK opsiyonlanabilir.
BA opsiyonlanabilir.
BKNG opsiyonlana